# Day 14 

In [293]:
import pandas as pd
import numpy as np
import sys
#np.set_printoptions(threshold=sys.maxsize)

In [294]:
with open('input_files/day14_input.txt') as f:
    lines = f.read().splitlines()

In [99]:
lines

['NNCB',
 '',
 'CH -> B',
 'HH -> N',
 'CB -> H',
 'NH -> C',
 'HB -> C',
 'HC -> B',
 'HN -> C',
 'NN -> C',
 'BH -> H',
 'NC -> B',
 'NB -> B',
 'BN -> B',
 'BB -> N',
 'BC -> B',
 'CC -> N',
 'CN -> C']

### Part 1

In [295]:
start_polymer = lines[0]
start_polymer

'OOFNFCBHCKBBVNHBNVCP'

In [296]:
polymer_template = [x.split(" -> ") for x in lines[2:]]
polymer_template

[['PH', 'V'],
 ['OK', 'S'],
 ['KK', 'O'],
 ['BV', 'K'],
 ['CV', 'S'],
 ['SV', 'C'],
 ['CK', 'O'],
 ['PC', 'F'],
 ['SC', 'O'],
 ['KC', 'S'],
 ['KF', 'N'],
 ['SN', 'C'],
 ['SF', 'P'],
 ['OS', 'O'],
 ['OP', 'N'],
 ['FS', 'P'],
 ['FV', 'N'],
 ['CP', 'S'],
 ['VS', 'P'],
 ['PB', 'P'],
 ['HP', 'P'],
 ['PK', 'S'],
 ['FC', 'F'],
 ['SB', 'K'],
 ['NC', 'V'],
 ['PP', 'B'],
 ['PN', 'N'],
 ['VN', 'C'],
 ['NV', 'O'],
 ['OV', 'O'],
 ['BS', 'K'],
 ['FP', 'V'],
 ['NK', 'K'],
 ['PO', 'B'],
 ['HF', 'H'],
 ['VK', 'S'],
 ['ON', 'C'],
 ['KH', 'F'],
 ['HO', 'P'],
 ['OO', 'H'],
 ['BC', 'V'],
 ['CS', 'O'],
 ['OC', 'B'],
 ['VB', 'N'],
 ['OF', 'P'],
 ['FK', 'H'],
 ['OH', 'H'],
 ['CF', 'K'],
 ['CC', 'V'],
 ['BK', 'O'],
 ['BH', 'F'],
 ['VV', 'N'],
 ['KS', 'V'],
 ['FO', 'F'],
 ['SH', 'F'],
 ['OB', 'O'],
 ['VH', 'F'],
 ['HH', 'P'],
 ['PF', 'C'],
 ['NF', 'V'],
 ['VP', 'S'],
 ['CN', 'V'],
 ['SK', 'O'],
 ['FB', 'S'],
 ['FN', 'S'],
 ['BF', 'H'],
 ['FF', 'V'],
 ['CB', 'P'],
 ['NN', 'O'],
 ['VC', 'F'],
 ['HK', 'F'],
 ['BO'

In [156]:
import regex as re
        
def run_one_step(start_polymer, polymer_template):
    temporary_polymer = start_polymer
    new_char_dict = {}
    for pair in polymer_template:
        #print("checking pair", pair)
        check_pair = pair[0]
        if check_pair in temporary_polymer:
            element = pair[1]
            matches_positions = [match.start() for match in re.finditer('(?='+check_pair+')', temporary_polymer)]
            #print("found ", check_pair, "add", pair[1], "in", matches_positions)
            for match in matches_positions:
                new_char_dict[match+1] = element
    
    #print(new_char_dict)
    i=0 #n. chars added
    for pos in sorted(new_char_dict):
        #print("Adding", new_char_dict[pos],"in",pos,"...")
        temporary_polymer = temporary_polymer[:pos+i]+new_char_dict[pos]+temporary_polymer[pos+i:]
        i +=1
        #print("temporary", temporary_polymer)
    final_polymer = temporary_polymer
    return final_polymer

In [157]:
def run_polymerization(start_polymer, polymer_template, n_steps):
    print(start_polymer)
    for n in range(0, n_steps):
        #print("start step", n+1, start_polymer)
        start_polymer = run_one_step(start_polymer, polymer_template)
        #print("after step", n+1, start_polymer)
    return start_polymer

In [177]:
final_polymer = run_polymerization(start_polymer, polymer_template, 2)
final_polymer

NNCB


'NBCCNBBBCBHCB'

In [178]:
Counter(final_polymer)

Counter({'N': 2, 'B': 6, 'C': 4, 'H': 1})

In [159]:
from collections import Counter
print(Counter(final_polymer).most_common(1)[0][1]-Counter(final_polymer).most_common()[-1][1])

1588


### Part 2

In [297]:
Counter(start_polymer)

Counter({'O': 2,
         'F': 2,
         'N': 3,
         'C': 3,
         'B': 4,
         'H': 2,
         'K': 1,
         'V': 2,
         'P': 1})

In [298]:
def initialize_counters(start_polymer):
    initial_counter=Counter()
    letters_counter=Counter(start_polymer)
    for pos in range(0, len(start_polymer)-1):
        initial_counter[start_polymer[pos:pos+2]]+=1
    return initial_counter, letters_counter

def run_one_step_fast(initial_counter,letters_counter, polymer_template):
    #print("Start", initial_counter,letters_counter)
    final_counter = initial_counter.copy()
    
    for pair in polymer_template:
        check_pair = pair[0]
        element = pair[1]
        #print("checking pair", check_pair)
        if initial_counter[check_pair]>=1:
            #print("reference counter", initial_counter)
            #print("pair found", check_pair, "new letter", element)
            final_counter[check_pair[0]+element]+=1*initial_counter[check_pair]
            final_counter[element+check_pair[1]]+=1*initial_counter[check_pair]
            final_counter[check_pair]-=1*initial_counter[check_pair]
            letters_counter[element]+=1*initial_counter[check_pair]
            
            #print("final", final_counter, letters_counter)  
    return final_counter,letters_counter

def run_polymerization_fast(start_counter,letters_counter, polymer_template, n_steps):
    for n in range(0, n_steps):
        start_counter,letters_counter = run_one_step_fast(start_counter,letters_counter, polymer_template)
        final_counter = start_counter
    return final_counter,letters_counter

In [302]:
initial_counter = initialize_counters(start_polymer)[0]
letters_counter = initialize_counters(start_polymer)[1]

final_counter, letters_counter = run_polymerization_fast(initial_counter,letters_counter, polymer_template, 40)

In [303]:
letters_counter

Counter({'O': 3130799559067,
         'F': 1367340055032,
         'N': 1079825178720,
         'C': 2243098080010,
         'B': 1852177036253,
         'H': 2025855409581,
         'K': 662215304710,
         'V': 2272753224170,
         'P': 4190532384255,
         'S': 2066124695947})

In [304]:
from collections import Counter
print(letters_counter.most_common(1)[0][1]-letters_counter.most_common()[-1][1])

3528317079545
